In [2]:
from transformers import AutoTokenizer,AutoModel
# from thuglm.modeling_chatglm import ChatGLMForConditionalGeneration
import torch
from peft import get_peft_model, LoraConfig, TaskType


In [3]:
model = AutoModel.from_pretrained(
    "yuanzhoulvpi/chatglm6b-dddd", trust_remote_code=True).half().cuda()

peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1,
    target_modules=['query_key_value',],
)
model = get_peft_model(model, peft_config)

# 在这里加载lora模型，注意修改chekpoint
peft_path = "test004/checkpoint-100/chatglm-lora.pt"
model.load_state_dict(torch.load(peft_path), strict=False)
model.eval()

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

c:\Users\yuanz\anaconda3\envs\mynet\lib\site-packages\peft\tuners\lora.py:173: UserWarning: fan_in_fan_out is set to True but the target module is not a Conv1D. Setting fan_in_fan_out to False.
  warnings.warn(


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): ChatGLMForConditionalGeneration(
      (transformer): ChatGLMModel(
        (word_embeddings): Embedding(150528, 4096)
        (layers): ModuleList(
          (0): GLMBlock(
            (input_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
            (attention): SelfAttention(
              (rotary_emb): RotaryEmbedding()
              (query_key_value): MergedLinear(
                in_features=4096, out_features=12288, bias=True
                (lora_dropout): Dropout(p=0.1, inplace=False)
                (lora_A): Linear(in_features=4096, out_features=16, bias=False)
                (lora_B): Conv1d(16, 8192, kernel_size=(1,), stride=(1,), groups=2, bias=False)
              )
              (dense): Linear(in_features=4096, out_features=4096, bias=True)
            )
            (post_attention_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
            (mlp): GLU(
              (d

In [4]:
tokenizer = AutoTokenizer.from_pretrained("yuanzhoulvpi/chatglm6b-dddd", trust_remote_code=True)

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


In [5]:
text ="为什么冰红茶和柠檬茶的味道一样？"

with torch.autocast("cuda"):
    res, history = model.chat(tokenizer=tokenizer, query=text,max_length=300)
    print(res)

冰红茶和柠檬茶的味道并不完全相同,虽然它们都是茶饮料,但它们的味道是由不同的茶叶、添加剂和糖精制而成的。

冰红茶是一种茶饮料,由红茶、糖精、水和其他添加剂制成。它的味道通常比柠檬茶更甜和浓郁。

柠檬茶是一种混合了柠檬和茶的饮料,它的味道主要由柠檬汁、茶和糖精制成。由于柠檬和茶都含有大量的天然调味品,所以它们的味道相似。
